In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import make_smoothing_spline

In [41]:
data_dict = {
    "B_wet_ground":{
        "conductivity":"../data/ITU-R_P.527-3_extracted_data/B_wet_ground_con.csv",
        "permittivity":"../data/ITU-R_P.527-3_extracted_data/B_wet_ground_per.csv",
        "sionna_func_name":"wet_ground_properties"
    },
    "D_medium_dry_ground":{
        "conductivity":"../data/ITU-R_P.527-3_extracted_data/D_medium_dry_ground_con.csv",
        "permittivity":"../data/ITU-R_P.527-3_extracted_data/D_medium_dry_ground_per.csv",
        "sionna_func_name":"medium_dry_ground_properties"
    },
    "E_very_dry_ground":{
        "conductivity":"../data/ITU-R_P.527-3_extracted_data/E_very_dry_ground_con.csv",
        "permittivity":"../data/ITU-R_P.527-3_extracted_data/E_very_dry_ground_per.csv",
        "sionna_func_name":"very_dry_ground_properties"
    },
    
}

In [42]:
from  sionna.rt import itu_materials

import ast
import inspect

# Get the source code of the outer function
source_code = inspect.getsource(itu_materials)

# Parse it into an AST (Abstract Syntax Tree)
parsed_code = ast.parse(source_code)

# Dictionary to store extracted inner functions
inner_functions = {}

# Find the outer function definition (instantiate_itu_materials)
for node in parsed_code.body:
    if isinstance(node, ast.FunctionDef) and node.name == "instantiate_itu_materials":
        # Traverse its body to find nested functions
        for inner_node in node.body:
            if isinstance(inner_node, ast.FunctionDef):  # Check for function definitions inside
                function_name = inner_node.name
                function_code = ast.unparse(inner_node)  # Convert AST node back to Python code

                # Execute the extracted function to define it in global scope
                exec(function_code, globals())

                # Store the function reference
                inner_functions[function_name] = globals()[function_name]

# Now, you can access the inner function dynamically
if "concrete_properties" in inner_functions:
    concrete_properties = inner_functions["concrete_properties"]
    print(concrete_properties(1e9))  # Example usage


(5.24, 0.0462)


In [45]:
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Generate logarithmically spaced x-axis values
x = np.logspace(-2, 6, 100)  # X-axis from 10^-2 to 10^6

# Generate y-axis values for permittivity and conductivity
y1 = np.logspace(-1, 3, 100)  # Y1 from 10^-1 to 10^3 (Relative Permittivity)
y2 = np.logspace(-5, 2, 100)  # Y2 from 10^-5 to 10^2 (Conductivity)

# Define fixed y-limits for both plots
y1_min, y1_max = np.min(y1), np.max(y1)  # Y1 range (Permittivity)
y2_min, y2_max = np.min(y2), np.max(y2)  # Y2 range (Conductivity)

# Compute log-scale height ratios to ensure same tick spacing
log_range_1 = np.log10(y1_max) - np.log10(y1_min)
log_range_2 = np.log10(y2_max) - np.log10(y2_min)
height_ratios = [log_range_1, log_range_2]

# Create a figure with two vertically stacked subplots
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 14),
                               gridspec_kw={'height_ratios': height_ratios, 'hspace': 0.04})

# Set both axes to logarithmic scale
ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_xscale('log')
ax2.set_yscale('log')

# Iterate through different ground types and plot data
for ground_type in file_path_dict.keys():

    sionna_x = np.linspace(1e9, 10e9, 100000)  # Generate x values
    
    # Obatin the Sionna's conductivity and permittivity data
    sionna_material_func = None
    if data_dict[ground_type]["sionna_func_name"] in inner_functions:
        sionna_material_func = inner_functions[data_dict[ground_type]["sionna_func_name"]]
    
    sionna_per, sionna_con = zip(*[ sionna_material_func(f) for f in sionna_x])

    ax1.plot(sionna_x/1e6, sionna_per, label="{} Sionna".format(ground_type), marker='*', markersize=1)
    ax2.plot(sionna_x/1e6, sionna_con, label="{} Sionna".format(ground_type), marker='*', markersize=1)
    
    # Read and process conductivity data
    df_con = pd.read_csv(data_dict[ground_type]["conductivity"], header=None, names=["x", "y"])
    df_con = df_con.sort_values(by="x")
    x_smooth = np.linspace(df_con["x"].min(), df_con["x"].max(), 100000)  # Generate smooth x values
    
    # Fit a smoothing spline for conductivity
    spl_con = make_smoothing_spline(df_con["x"], df_con["y"])
    
    # Plot raw conductivity data and smoothed curve
    ax2.scatter(df_con["x"], df_con["y"], label="{} Raw Data".format(ground_type), alpha=0.1)
    ax2.plot(x_smooth, spl_con(x_smooth), label="{} make_smoothing_spline".format(ground_type), linestyle="-")
    
    # Read and process permittivity data
    df_per = pd.read_csv(data_dict[ground_type]["permittivity"], header=None, names=["x", "y"])
    df_per = df_per.sort_values(by="x")
    
    # Fit a smoothing spline for permittivity
    spl_per = make_smoothing_spline(df_per["x"], df_per["y"])
    
    # Plot raw permittivity data and smoothed curve
    ax1.scatter(df_per["x"], df_per["y"], label="{} Raw Data".format(ground_type), alpha=0.2)
    ax1.plot(x_smooth, spl_per(x_smooth), label="{} make_smoothing_spline".format(ground_type), linestyle="-")
    
# Apply y-limits to each subplot
ax1.set_ylim(y1_min, y1_max)
ax2.set_ylim(y2_min, y2_max)
ax2.set_xlim(np.min(x), np.max(x))  # Apply shared x-axis range

# Enable grid on both subplots
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
ax2.grid(True, which='both', linestyle='--', linewidth=0.5)

# Set axis labels and titles
ax1.set_ylabel(r"Relative permittivity, $\epsilon_{r}$")
ax2.set_ylabel(r"Conductivity, $\sigma$ (S/m)")
ax2.set_xlabel("Frequency (MHz)")

# Add legends to both subplots
ax1.legend()
ax2.legend()

# Align y-labels for better readability
fig.align_ylabels()

# Define image file paths
image_path = "../data/ITU-R_P.527-3_extracted_data/ITU-527-3.png"  # Path to reference image
plot_path = "plot_temp.png"  # Temporary file for saved plot

# Save the generated plot as an image
fig.savefig(plot_path, bbox_inches='tight', pad_inches=0)
plt.close(fig)  # Close figure to prevent redundant display

# Generate an HTML layout to display the saved plot and reference image side by side
html_code = f"""
<div style="display: flex; justify-content: center; gap: 20px;">
    <div>
        <img src="{plot_path}" style="height: 70vh; width: auto;">
    </div>
    <div>
        <img src="{image_path}" style="height: 70vh; width: auto;">
    </div>
</div>
"""

# Display the images in Jupyter Notebook
display(HTML(html_code))


(5.24, 0.0462)
